<a href="https://colab.research.google.com/github/Ivan412/training/blob/main/6_python_advaced/4_reports%20and%20automation/task4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Продвинутый Python. Часть 4: Отчеты и автоматизация. Отправка email и интеграция

---


#**Задание: автоматические отчеты**

Соберите отчет по результатам ЕГЭ в 2018-2019 году, используя данные

https://video.ittensive.com/python-advanced/data-9722-2019-10-14.utf.csv

и отправьте его в HTML формате по адресу support@ittensive.com, используя только Python.

В отчете должно быть:
*   общее число отличников (учеников, получивших более 220 баллов по ЕГЭ в Москве),
*   распределение отличников по округам Москвы,
*   название школы с лучшими результатами по ЕГЭ в Москве.


Диаграмма распределения должна быть вставлена в HTML через data:URI формат (в base64-кодировке).

Дополнительно: приложите к отчету PDF документ того же содержания (дублирующий письмо).

In [ ]:
%%capture
#установим wkhtmltopdf
! wget "https://github.com/wkhtmltopdf/wkhtmltopdf/releases/download/0.12.4/wkhtmltox-0.12.4_linux-generic-amd64.tar.xz" && \
  tar vxfJ "wkhtmltox-0.12.4_linux-generic-amd64.tar.xz" && \
  mv wkhtmltox/bin/wkhtmltopdf /usr/bin/wkhtmltopdf
!pip install pdfkit

In [ ]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import io
import binascii
from jinja2 import Template
import smtplib
from email import encoders
from email.mime.text import MIMEText
from email.mime.base import MIMEBase
from email.mime.multipart import MIMEMultipart
import pdfkit
from getpass import getpass
plt.ioff()
config = pdfkit.configuration(wkhtmltopdf="/usr/bin/wkhtmltopdf")

In [ ]:
#html шаблон отчета
TEMPLATE = '''<!DOCTYPE html>
<html>
<head>
</head>
  <title>Отчет по результатам ЕГЭ в 2018-2019 году</title>
  <meta charset="utf-8"/>
<body>
  <h1>Отчет по результатам ЕГЭ в 2018-2019 году</h1>
  <h2>Общее число отличников</h2>
  <p>Общее число отличников в школах москвы за 2018-2019г: {{data.total_over220}}</p>
  <h2>Распределение отличников по округам Москвы</h2>
  <img src="{{data.image}}" alt="Распределение отличников по округам Москвы">
  <h2>Школа с лучшими результатами по ЕГЭ в Москве</h2>
  <p>Школа с наибольшим числом отличников за 2018-2019г:<br>{{data.best_scool}}</p>
</body>
</html>'''

In [ ]:
#загрузим данные необходимык для выполнения задания
cols = ['EDU_NAME', 'AdmArea', 'PASSES_OVER_220', 'YEAR']
data = pd.read_csv('https://video.ittensive.com/python-advanced/data-9722-2019-10-14.utf.csv', delimiter=';', usecols=cols)
#оставим данные за 2018-2019г
data = data.loc[data['YEAR'] == '2018-2019'][['EDU_NAME', 'AdmArea', 'PASSES_OVER_220']]

In [ ]:
#посчитаем количество отличников и найдем лучшую школу(критерий - самое большое число отличников)
total_over220 = data['PASSES_OVER_220'].sum()
best_scool = data.sort_values(by=['PASSES_OVER_220'], ascending=False).iloc[0]['EDU_NAME']

In [ ]:
#подготовим данные
alloc_over220 = data.groupby('AdmArea').sum()
#нарисуем круговую диаграмму
fig = plt.figure(figsize=(12, 12))
ax = fig.add_subplot(1, 1, 1)
alloc_over220['PASSES_OVER_220'].plot(ax=ax, kind='pie', autopct='%.2f', pctdistance=0.95, labels=None)
ax.set_xlabel('')
ax.set_ylabel('')
plt.legend(labels=alloc_over220.index, ncol=3)
#сохраним диаграмму в bin2ascii формате
img_buf = io.BytesIO()
plt.savefig(img_buf, format='png', bbox_inches="tight")
img_buf.seek(0)
img = 'data:image/png;base64,' + binascii.b2a_base64(img_buf.read(), newline=False).decode("UTF-8")
del(img_buf)

In [ ]:
#отрендерим шаблон и создадим pdf 
html = Template(TEMPLATE).render(data={'image': img, 'total_over220': total_over220, 'best_scool': best_scool})
options = {'page-size': 'A4', 'header-right': '[page]'}
pdf_buf = io.BytesIO()
pdf_buf.write(pdfkit.from_string(html, configuration=config, options=options))

63845

In [ ]:
#подключимся к серверу
server = smtplib.SMTP_SSL("smtp.yandex.com", 465)
login = input ("Введите login: ")
password = getpass("Введите password: ")
From = input ("Введите адрес отправителя: ")
To = input ("Введите адрес получателя: ")
server.login(login, password)
#сфоримруем текстовую часть письма
letter = MIMEMultipart()
letter["From"] = From
letter["Subject"] = "Продвинутый Python. Часть 4. Задание - автоматические отчеты"
letter["Content-Type"] = "text/html; charset=utf-8"
letter["To"] = To
letter.attach(MIMEText(html, 'html'))
#сфоримруем вложение и прекрепим к письму
attachement = MIMEBase('application', 'pdf')
pdf_buf.seek(0)
attachement.set_payload(pdf_buf.read())
attachement.add_header('Content-Disposition', 'attachement; filename="scool-stats.pdf"')
encoders.encode_base64(attachement)
letter.attach(attachement)
server.sendmail(From, To, letter.as_string())
del(password)